## Assemble Matrices Script

This script will assemble the two stiffness matrices that we need for the analysis of the spectrum of
$$ -\nabla^2 u = \omega^2 u $$
on our (thin-structure) cross domain $\Omega$.
The weak form of this equation is
$$ \int_{\Omega} \nabla u \cdot \nabla \phi - \omega^2 u \phi \ \mathrm{d}x = 0.$$

This domain consists of $N$ period cells along each axis (for a total of $N^2$ cells), each of size $1\times 1$.
Tube widths are taken as $h = \frac{1}{2N}$, whilst the radii of the circles is $r = \sqrt{\frac{h}{\alpha\pi}}$, where $\alpha\in\mathbb{R}$ is (a proxy for) the coupling constant which appears in the singular-structure limit problem.
**NOTE:** This assumes $\alpha \approx \frac{V_{edge}}{V_{vertex}}$ - this fraction may be reversed in other places of our work!

What will be done is the stiffness matrices $A_1$ and $A_2$ for $\int_{\Omega} \nabla u \cdot \nabla \phi \ \mathrm{d}x$ and $\int_{\Omega} u \phi \ \mathrm{d}x$ (respectively) will be assembled and saved.
Then, we can pass these to another script to compute the eigenvalues $\omega^2$ of the generalised eigenvalue problem
$$ \left( A_1 + \omega^2 A_2 \right)u = 0,$$
and unpack the solutions $u$.
**NOTE:** The $-$ sign is _not_ included in the stiffness matrix for $A_2$!

### Requirements:
- This script requires the fenicsproject environment to be active
- This script requires the following subdirectories be available: `./StiffnessMatrixDump/`

### Outputs:
- Stiffness matrix files stored in the directory `./StiffnessMatrixDump/`, under the naming convention `a1_N*.npz` and `a2_N*.npz` where `*` is replaced by the value of $N$ being used.

In [1]:
from fenics import *
from dolfin import *
import scipy.sparse as sp

# This is the name of our mesh files, minus the extension.
filePrefix = 'FEM_CrossGraphSetup'
# This is the gmsh file which generated our domain's mesh. We can infer information about our domain from it
gmshFile = filePrefix + '.geo'
# This is the folder into which we will place our stiffness matrices, once they are assembled
matDumpFolder = 'StiffnessMatrixDump'

# Deduce value of N from gmshFile - it appears on line 12,
# with the 5th character being (the start of) the value of N.
with open('FEM_CrossGraphSetup.geo') as fp:
    for i, line in enumerate(fp):
        if i == 11:
            # 11th line
            Nstr = line[4:-2] #take all characters after the numerical value appears in the file
        elif i > 11:
            #terminate loop early
            break
# We want N as a float because we will use it in computations, to define points on meshes etc. Int might cause bugs.
for i, char in enumerate(Nstr):
    if char==';':
        Nstr = Nstr[:i]
        break
N = float(Nstr)

# Create filenames to save outputs to
fileA1Name = './' + matDumpFolder + '/a1_N' + Nstr
fileA2Name = './' + matDumpFolder + '/a2_N' + Nstr

In [2]:
# create a subclass for the 2D-periodic domain...
class PeriodicDomain(SubDomain):
    #map left--> right and bottom-->top

    def inside(self, x, on_boundary):
        # return True if on left or bottom boundary AND NOT on one of the two corners
        return bool((near(x[0], 0) or near(x[1], 0)) and
                (not ((near(x[0], 0) and near(x[1], N)) or
                        (near(x[0], N) and near(x[1], 0)))) and on_boundary)

    def map(self, x, y):
        if near(x[0], N) and near(x[1], N):
            y[0] = x[0] - N
            y[1] = x[1] - N
        elif near(x[0], N):
            y[0] = x[0] - N
            y[1] = x[1]
        else:   # near(x[1], N)
            y[0] = x[0]
            y[1] = x[1] - N
#concludes the SubDomain subclass definition

# Now we import our mesh...
meshFile = filePrefix + '.xml'
mesh = Mesh(meshFile)

# Create function space for the problem, 
# making sure we tell the function space that the domain is periodic
V = FunctionSpace(mesh, 'CG', 1, constrained_domain=PeriodicDomain())

# Define variational problem for A_1 and A_2
u = TrialFunction(V)
v = TestFunction(V)
A1 = dot(grad(u), grad(v))*dx
A2 = u*v*dx

In [3]:
# Now build the stiffness matrices A_1 and A_2.
# Save these to files in CSR format since CSC isn't available
# To aid with memory problems, we build one at a time and overwrite the same variable after writing to a file
# Build A_1
A = assemble(A1)
A_mat = as_backend_type(A).mat()
A_sparray = sp.csr_matrix(A_mat.getValuesCSR()[::-1], shape = A_mat.size) #store in sparse format
sp.save_npz(fileA1Name, A_sparray)
# Now build A_2
A = assemble(A2)
A_mat = as_backend_type(A).mat()
A_sparray = sp.csr_matrix(A_mat.getValuesCSR()[::-1], shape = A_mat.size) #store in sparse format
sp.save_npz(fileA2Name, A_sparray)